In [1]:
# If you're in Colab...
import os, sys

    
    # Pull files from Github repo

!git init .
!git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
!git pull origin master
    
    # Change into directory for module
os.chdir('module1')

Reinitialized existing Git repository in /Users/maximevacher-materno/Downloads/.git/
fatal: remote origin already exists.
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Total 13 (delta 8), reused 8 (delta 8), pack-reused 5
Unpacking objects: 100% (13/13), done.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
   f5efe86..8df744a  master     -> origin/master
Updating f5efe86..8df744a
Fast-forward
 module3/assignment_kaggle_challenge_3.ipynb | 176 ++++++
 module3/lesson_kaggle_challenge_3.ipynb     | 802 ++++++++++++++++++++++++++++
 2 files changed, 978 insertions(+)
 create mode 100644 module3/assignment_kaggle_challenge_3.ipynb
 create mode 100644 module3/lesson_kaggle_challenge_3.ipynb


In [18]:
import numpy as np
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings 
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [19]:
train, validate = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)
train.shape, validate.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [20]:
train.date_recorded = pd.to_datetime(train.date_recorded)

In [21]:
import numpy as np

def clean(X):
    # stop SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'district_code']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)

    # quantity & quantity_group are duplicates
    X = X.drop(columns=['quantity_group', 'installer', 'extraction_type_group', 
                        'extraction_type_class', 'payment_type', 'waterpoint_type_group'])
   

    #convert date_recorded 
    X['date_recorded'] = pd.to_datetime(X.date_recorded)

    X['year_recorded'] = X['date_recorded'].dt.year

    X = X.drop(columns='date_recorded')

    # #create a new feature for pump_age
    X['pump_age'] = X['year_recorded'] - X.construction_year
    
    # # replace negative pump ages with nan 
    # # which also decreased validation accuracy slightly
    X['pump_age'] = X['pump_age'].replace([-7, -6, -5, -4, -3, -2, -1], np.nan)

    # return the wrangled dataframe
    return X

train = clean(train)
val = clean(validate)
test = clean(test)

In [22]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'year_recorded', 'pump_age', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type', 'management', 'management_group', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type']


In [23]:
# The status_group column is the target
target = 'status_group'

X_train = train.drop(columns=target)
y_train = train[target]
X_validate = val.drop(columns=target)
y_validate = val[target]
X_test = test

In [27]:
##XGB Classifier

from xgboost import XGBClassifier

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# fit on train, score on val
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['funder', 'wpt_name', 'basin',
                                      'subvillage', 'region', 'lga', 'ward',
                                      'public_meeting', 'recorded_by',
                                      'scheme_management', 'scheme_name',
                                      'permit', 'extraction_type', 'management',
                                      'management_group', 'payment',
                                      'water_quality', 'quality_group',
                                      'quantity', 'source', 'source_type',
                                      'source_class', 'wate...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
              

In [25]:
print('Validation Accuracy: ', pipeline.score(X_validate, y_validate))

Validation Accuracy:  0.7438552188552189


In [16]:
import numpy as np
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings 
warnings.filterwarnings(action='ignore', category=FutureWarning, module='xgboost')

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    XGBRegressor(objective='reg:squarederror', random_state=42)
)

param_distributions = {
    'xgbregressor_n_estimator': randint(100, 1000),
    'xgbregressor_max_depth': randint(3, 10),
    'xgbregressor_learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions= param_distributions,
    n_iter=5,
    cv=2,
    scoring='neg_mean_absolute_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train)


# print('Best Hyperparameters', search.best_params_)
# print('Cross-Validation MAE', -search.best_score)


Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    2.9s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    2.9s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    2.9s remaining:    1.2s


ValueError: Invalid parameter xgbregressor_learning_rate for estimator Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=None, drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=None, return_df=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, boos...
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0,
                              importance_type='gain', learning_rate=0.1,
                              max_delta_step=0, max_depth=3, min_child_weight=1,
                              missing=nan, n_estimators

In [ ]:
# pip install xgboost